In [1]:
import numpy as np
import cvxpy as cp
import scipy 
import generate_constant_matrices as gen_mat
import generate_proj_mat as proj
import scipy.sparse as sparse
import generate_matrices_SDP as SPD_gen
import matplotlib.pyplot as plt

np.random.seed(0)

In [24]:
# If we allow beta to be any real matrix, there will always be a value of beta that will drive the n**2 - tr(J_dagger J) to -infty
N_omega = 21
n = 0.1
omega = np.linspace(-3, 3, N_omega)
beta = scipy.linalg.hankel(np.random.random(N_omega), np.random.random(N_omega))
delta_k = 1.j*np.diag(omega)
Q_plus = delta_k + beta
Q_minus = delta_k - beta
N_z = 5
z = np.linspace(0, 0.001, N_z)
delta_z = np.abs(z[1] - z[0])
W_plus = [scipy.linalg.expm(Q_plus*z[i]) for i in range(N_z)]
W_minus = [scipy.linalg.expm(Q_minus*z[i]) for i in range(N_z)]
J = [0.25*(W_plus[i]@W_plus[i].conj().T + W_minus[i]@W_minus[i].conj().T - 2*np.eye(N_omega)) for i in range(N_z)]
X = np.concatenate((np.concatenate(J + W_plus + W_minus), beta))
A = np.random.random((N_omega, N_omega))
A = A@A.conj().T
eig_val, sing_eigvec = np.linalg.eig(A)
Z = sing_eigvec@sing_eigvec.conj().T
W = np.concatenate([X, Z])
Y = W@W.conj().T
# Check for accuracy of finite difference/Green function num approx
# N_z = 20, N_omega = 101 for z going from 0 to 0.01 and omega from -3 to 3 with delta_v = 1 seems ok
# Try expressing SDP in terms of degrees of freedom

In [25]:
projection = np.zeros((N_omega, N_omega))
projections = []
for i in range(N_omega):
    for j in range(N_omega):
        proj_copy = projection.copy()
        proj_copy[i, j] = 1
        projections.append(sparse.csc_matrix(proj_copy))

In [26]:
N_matrices, kron_delta = SPD_gen.generate_constr_Z_mat(N_omega, N_z)

In [27]:
constr_list = []
projection = sparse.eye(N_omega)
quad_J_constr, photon_nbr_constr  = SPD_gen.generate_base_matrices(N_omega, N_z, n)
constr_list.append(photon_nbr_constr)
constr_list += quad_J_constr
beta_quad_constr = SPD_gen.generate_beta_cov_constraint(omega, N_z, 20)
constr_list.append(beta_quad_constr)
real_plus_M_mat_list = []
imag_plus_M_mat_list = []
real_minus_M_mat_list = []
imag_minus_M_mat_list = []
J_def_list = []
herm_mat_list = []
for i in range(len(projections)):
    real_plus_M_mat, imag_plus_M_mat, real_minus_M_mat, imag_minus_M_mat = SPD_gen.generate_dynamics_matrices(omega, z, projections[i])
    J_def = SPD_gen.generate_def_J(N_omega, N_z, projections[i])
    herm_mat = SPD_gen.generate_matrices_herm(N_omega, N_z, projections[i])
    real_plus_M_mat_list += real_plus_M_mat
    imag_plus_M_mat_list += imag_plus_M_mat
    real_minus_M_mat_list += real_minus_M_mat
    imag_minus_M_mat_list += imag_minus_M_mat
    J_def_list += J_def
    herm_mat_list += herm_mat
constr_list += real_minus_M_mat_list + imag_plus_M_mat_list + real_minus_M_mat_list + imag_minus_M_mat_list + J_def_list + herm_mat_list

In [28]:
obj_f_quad = -gen_mat.get_proj_quad_diag(N_omega, N_z, projection)[N_z - 1]
obj_mat = sparse.bmat([[obj_f_quad, sparse.csc_matrix(((3*N_z + 1)*N_omega, N_omega))],[sparse.csc_matrix((N_omega, (3*N_z + 1)*N_omega)), ((n**2)/N_omega)*sparse.eye(N_omega)]])

In [29]:
X = cp.Variable(((3*N_z + 2)*N_omega, (3*N_z + 2)*N_omega), complex=True)
constraints = [X >> 0]
constraints += [cp.real(cp.trace(quad_J_constr[i]@X)) <= 0 for i in range(N_z)]
constraints += [cp.real(cp.trace(herm_mat_list[i]@X)) == 0 for i in range(N_z)]
constraints += [cp.real(cp.trace(real_plus_M_mat_list[i]@X)) == 0 for i in range(len(real_minus_M_mat_list))]
constraints += [cp.real(cp.trace(imag_plus_M_mat_list[i]@X)) == 0 for i in range(len(real_minus_M_mat_list))]
constraints += [cp.real(cp.trace(real_minus_M_mat_list[i]@X)) == 0 for i in range(len(real_minus_M_mat_list))]
constraints += [cp.real(cp.trace(imag_minus_M_mat_list[i]@X)) == 0 for i in range(len(real_minus_M_mat_list))]
constraints += [cp.real(cp.trace(N_matrices[i]@X)) == kron_delta[i] for i in range(len(N_matrices))]
constraints.append(cp.real(cp.trace(beta_quad_constr@X)) <= 0)
constraints.append(cp.real(cp.trace(photon_nbr_constr@X)) == 0)
problem = cp.Problem(cp.Minimize(cp.real(cp.trace(obj_mat@X))), constraints)

In [30]:
problem.solve(verbose = True, max_iters = int(10000))

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Feb 02 03:14:50 PM: Your problem has 127449 variables, 9064 constraints, and 0 parameters.
(CVXPY) Feb 02 03:14:51 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 02 03:14:51 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 02 03:14:51 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 02 03:14:51 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 02 03:14:51 PM: Compiling problem (target solver=SCS)

(CVXPY) Feb 02 03:15:50 PM: Applying reduction SCS
(CVXPY) Feb 02 03:15:50 PM: Finished problem compilation (took 5.958e+01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Feb 02 03:15:50 PM: Invoking solver SCS  to obtain a solution.
------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 254898, constraints m: 264318
cones: 	  z: primal zero / dual free vars: 9057
	  l: linear vars: 6
	  s: psd vars: 255255, ssize: 1
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 10000, normalize: 1, rho_x: 1.00e-06
	  

/home/jiaqizhao/miniconda3/envs/cvxpy_opt/lib/python3.11/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


0.0004873224795089691

In [32]:
after_opt_thingy = np.array(X.value)

In [33]:
test_constr_values = [np.trace(constr_list[i]@after_opt_thingy) for i in range(len(constr_list))]
true_constr_values = [np.trace(constr_list[i]@Y) for i in range(len(constr_list))]

In [16]:
len(real_plus_M_mat_list), len(imag_plus_M_mat_list), len(real_minus_M_mat_list), len(imag_minus_M_mat_list), len(J_def_list), len(herm_mat_list)

(605, 605, 605, 605, 605, 605)

In [34]:
[np.trace(J_def_list[i]@after_opt_thingy) for i in range(len(J_def_list))]

[(1.035685468634618-3.8676186949886236e-25j),
 (1.0356915914735407+2.3680388419605086e-25j),
 (1.0356511130479726-4.93096742492969e-25j),
 (1.0356241277588512+5.929892450080043e-25j),
 (1.0165630131042689-2.995258167111172e-21j),
 (2.4097315689046092e-14-9.832121010614019e-14j),
 (3.3805032603044813e-13-1.8864342587310234e-13j),
 (-2.4244112068149764e-13-1.2691169362007525e-14j),
 (-3.29425453174961e-13-2.0961813698819493e-13j),
 (-5.5900080275973406e-14-1.507776087723919e-13j),
 (9.07129279026702e-14+5.648113943222877e-15j),
 (2.1354665965269562e-13-6.231193070211845e-14j),
 (6.898831455387115e-14+1.573877223022571e-14j),
 (1.4180220929844275e-13+4.5220292854918e-15j),
 (9.615111599531468e-15-3.4546192527152115e-14j),
 (2.0300888170139012e-13-2.058879272601784e-13j),
 (1.430177051957721e-13-1.1785158374572594e-13j),
 (1.3179244526596197e-14-1.364342760225108e-13j),
 (3.762913573120032e-13-5.411142570023309e-14j),
 (1.897056569661731e-13-2.1351892967239287e-13j),
 (-5.126997973435799e-

In [13]:
[i for i in range(len(constr_list)) if np.abs(np.trace(constr_list[i]@after_opt_thingy)) > 10**-5]

[1,
 2,
 3,
 4,
 5,
 6,
 2427,
 2428,
 2429,
 2430,
 2431,
 2487,
 2488,
 2489,
 2490,
 2491,
 2547,
 2548,
 2549,
 2550,
 2551,
 2607,
 2608,
 2609,
 2610,
 2611,
 2667,
 2668,
 2669,
 2670,
 2671,
 2727,
 2728,
 2729,
 2730,
 2731,
 2787,
 2788,
 2789,
 2790,
 2791,
 2847,
 2848,
 2849,
 2850,
 2851,
 2907,
 2908,
 2909,
 2910,
 2911,
 2967,
 2968,
 2969,
 2970,
 2971,
 3027,
 3028,
 3029,
 3030,
 3031]

In [66]:
np.trace(J[-1]) - n

(-0.09994682506711763+0j)

In [63]:
true_constr_values[:10], test_constr_values[:10]

([(-0.09994682506711766+2.938735877055719e-39j),
  (-0.010000000000000002+0j),
  (-0.00999999999290457+0j),
  (-0.00999999988647252+0j),
  (-0.009999999425262328+0j),
  (-0.009999998183523932+0j),
  (-301.7561309408215+0j),
  (-3.9968028886505635e-15+0j),
  (-3.698608086466493e-11+0j),
  (-7.389910905430952e-11+0j)],
 [(3.518127820317796e-08-5.204423489997715e-22j),
  (-0.003011872581844907+0j),
  (-0.0030118725815929106+0j),
  (-0.0030118725814787762+0j),
  (-0.003011872581614068+0j),
  (-0.0015449695898165257+0j),
  (-249.0354517012688+0j),
  (-4.940492459581947e-13-2.389798526853387e-21j),
  (-4.958256027975949e-13-3.7688922078712994e-13j),
  (-4.947153797729698e-13-8.528761464426937e-13j)])

In [47]:
constr_values = [constraints[i].value() for i in range(len(constraints))]
[i for i in range(len(constr_values)) if constr_values[i] == False], len(constr_values)

([2498], 2499)